# Tutorial 06, case 6: Navier-Stokes problem with distributed control

In this tutorial we solve the optimal control problem

$$\min J(y, u) = \frac{1}{2} \int_{\Omega} |v - v_d|^2 dx + \frac{\alpha}{2} \int_{\Omega} |u|^2 dx$$
s.t.
$$\begin{cases}
- \nu \Delta v + v \cdot \nabla v + \nabla p = f + u   & \text{in } \Omega\\
                                \text{div} v = 0       & \text{in } \Omega\\
                                           v = 0       & \text{on } \partial\Omega
\end{cases}$$

where
$$\begin{align*}
& \Omega                      & \text{unit square}\\
& u \in [L^2(\Omega)]^2       & \text{control variable}\\
& v \in [H^1_0(\Omega)]^2     & \text{state velocity variable}\\
& p \in L^2(\Omega)           & \text{state pressure variable}\\
& \alpha > 0                  & \text{penalization parameter}\\
& v_d                         & \text{desired state}\\
& \nu                         & \text{kinematic viscosity}\\
& f                           & \text{forcing term}
\end{align*}$$
using an adjoint formulation solved by a one shot approach

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
import sympy
from ufl import derivative, div, grad, inner, Measure, replace, TestFunction, TrialFunction
from dolfinx import Constant, DirichletBC, Function, FunctionSpace, VectorFunctionSpace
from dolfinx.fem import locate_dofs_topological
from dolfinx.io import XDMFFile
from dolfinx.plot import create_vtk_topology
from multiphenicsx.fem import (assemble_matrix_block, assemble_scalar, assemble_vector_block,
                               BlockVecSubVectorWrapper, create_vector_block, create_matrix_block)
import pyvista

### Mesh

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "data/square.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    mesh.topology.create_connectivity_all()
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    boundaries = infile.read_meshtags(mesh, name="boundaries")
boundaries_1234 = boundaries.indices[np.isin(boundaries.values, (1, 2, 3, 4))]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

In [ ]:
def dolfinx_to_pyvista_mesh(mesh):
    num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
    cell_entities = np.arange(num_cells, dtype=np.int32)
    pyvista_cells, cell_types = create_vtk_topology(mesh, mesh.topology.dim, cell_entities)
    grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, mesh.geometry.x)
    return grid

In [ ]:
def pyvista_mesh_plot(mesh):
    grid = dolfinx_to_pyvista_mesh(mesh)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_mesh_plot(mesh)

### Function spaces

In [ ]:
Y_velocity = VectorFunctionSpace(mesh, ("Lagrange", 2))
Y_pressure = FunctionSpace(mesh, ("Lagrange", 1))
U = VectorFunctionSpace(mesh, ("Lagrange", 2))
Q_velocity = Y_velocity.clone()
Q_pressure = Y_pressure.clone()

### Trial and test functions

In [ ]:
(dv, dp) = (TrialFunction(Y_velocity), TrialFunction(Y_pressure))
(w, q) = (TestFunction(Y_velocity), TestFunction(Y_pressure))
du = TrialFunction(U)
r = TestFunction(U)
(dz, db) = (TrialFunction(Q_velocity), TrialFunction(Q_pressure))
(s, d) = (TestFunction(Q_velocity), TestFunction(Q_pressure))

### Solution

In [ ]:
(v, p) = (Function(Y_velocity), Function(Y_pressure))
u = Function(U)
(z, b) = (Function(Q_velocity), Function(Q_pressure))

 ### Problem data

In [ ]:
alpha = 1.e-5
epsilon = 1.e-5
x, y = sympy.symbols("x[0], x[1]")
psi_d = 10 * (1 - sympy.cos(0.8 * np.pi * x)) * (1 - sympy.cos(0.8 * np.pi * y)) * (1 - x)**2 * (1 - y)**2
v_d_x = sympy.lambdify([x, y], psi_d.diff(y, 1))
v_d_y = sympy.lambdify([x, y], - psi_d.diff(x, 1))
v_d = Function(Y_velocity)
v_d.interpolate(lambda x: np.stack((v_d_x(x[0], x[1]), v_d_y(x[0], x[1])), axis=0))
nu = 0.01
ff = Constant(mesh, (0., 0.))
bc0 = Function(Y_velocity)

### Optimality conditions

In [ ]:
F = [nu * inner(grad(z), grad(w)) * dx + inner(grad(w) * v, z) * dx
     + inner(grad(v) * w, z) * dx - b * div(w) * dx + inner(v - v_d, w) * dx,
     - q * div(z) * dx + epsilon * b * q * dx,
     alpha * inner(u, r) * dx - inner(z, r) * dx,
     nu * inner(grad(v), grad(s)) * dx + inner(grad(v) * v, s) * dx - p * div(s) * dx - inner(u + ff, s) * dx,
     - d * div(v) * dx + epsilon * p * d * dx]
dF = [[derivative(F_i, u_j, du_j) for (u_j, du_j) in zip((v, p, u, z, b), (dv, dp, du, dz, db))] for F_i in F]
dF[3][3] = Constant(mesh, 0.) * inner(dz, s) * dx
bdofs_Y_velocity_1234 = locate_dofs_topological((Y_velocity, Y_velocity), mesh.topology.dim - 1, boundaries_1234)
bdofs_Q_velocity_1234 = locate_dofs_topological((Q_velocity, Y_velocity), mesh.topology.dim - 1, boundaries_1234)
bc = [DirichletBC(bc0, bdofs_Y_velocity_1234, Y_velocity),
      DirichletBC(bc0, bdofs_Q_velocity_1234, Q_velocity)]

### Cost functional

In [ ]:
J = 0.5 * inner(v - v_d, v - v_d) * dx + 0.5 * alpha * inner(u, u) * dx

### Class for interfacing with SNES

In [ ]:
class NonlinearBlockProblem(object):
    def __init__(self, F, dF, solutions, bcs):
        self._F = F
        self._dF = dF
        self._obj_vec = create_vector_block(F)
        self._solutions = solutions
        self._bcs = bcs

    def update_solutions(self, x):
        x.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
        with BlockVecSubVectorWrapper(x, [c.function_space.dofmap for c in self._solutions]) as x_wrapper:
            for x_wrapper_local, component in zip(x_wrapper, self._solutions):
                with component.vector.localForm() as component_local:
                    component_local[:] = x_wrapper_local

    def obj(self, snes, x):
        self.F(snes, x, self._obj_vec)
        return self._obj_vec.norm()

    def F(self, snes, x, F_vec):
        self.update_solutions(x)
        with F_vec.localForm() as F_vec_local:
            F_vec_local.set(0.0)
        assemble_vector_block(F_vec, self._F, self._dF, self._bcs, x0=x, scale=-1.0)

    def dF(self, snes, x, dF_mat, _):
        dF_mat.zeroEntries()
        assemble_matrix_block(dF_mat, self._dF, self._bcs, diagonal=1.0)
        dF_mat.assemble()

### Uncontrolled functional value

In [ ]:
# Create problem by extracting state forms from the optimality conditions
F_state = [replace(F[i], {s: w, d: q}) for i in (3, 4)]
dF_state = [[replace(dF[i][j], {s: w, d: q}) for j in (0, 1)] for i in (3, 4)]
bc_state = [bc[0]]
problem_state = NonlinearBlockProblem(F_state, dF_state, (v, p), bc_state)
F_vec_state = create_vector_block(F_state)
dF_mat_state = create_matrix_block(dF_state)

In [ ]:
# Solve
vp = create_vector_block([F[j] for j in (0, 1)])
snes = PETSc.SNES().create(mesh.mpi_comm())
snes.setTolerances(max_it=20)
snes.getKSP().setType("preonly")
snes.getKSP().getPC().setType("lu")
snes.getKSP().getPC().setFactorSolverType("mumps")
snes.setObjective(problem_state.obj)
snes.setFunction(problem_state.F, F_vec_state)
snes.setJacobian(problem_state.dF, J=dF_mat_state, P=None)
snes.setMonitor(lambda _, it, residual: print(it, residual))
snes.solve(None, vp)
problem_state.update_solutions(vp)  # TODO can this be safely removed?

In [ ]:
J_uncontrolled = mesh.mpi_comm().allreduce(assemble_scalar(J), op=MPI.SUM)
print("Uncontrolled J =", J_uncontrolled)
assert np.isclose(J_uncontrolled, 0.1784536)

In [ ]:
def pyvista_scalar_field_plot(mesh, scalar_field, name):
    grid = dolfinx_to_pyvista_mesh(mesh)
    grid.point_arrays[name] = scalar_field.compute_point_values()
    grid.set_active_scalars(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
def pyvista_vector_field_plot(mesh, vector_field, name, factor):
    grid = dolfinx_to_pyvista_mesh(mesh)
    values = np.zeros((mesh.geometry.x.shape[0], 3))
    values[:, :2] = vector_field.compute_point_values()
    grid.point_arrays[name] = values
    grid.set_active_vectors(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    glyphs = grid.glyph(orient=name, factor=factor)
    plotter.add_mesh(glyphs)
    plotter.show()

In [ ]:
pyvista_vector_field_plot(mesh, v, "uncontrolled state velocity", factor=1e-1)

In [ ]:
pyvista_scalar_field_plot(mesh, p, "uncontrolled state pressure")

### Optimal control

In [ ]:
# Create problem associated to the optimality conditions
problem = NonlinearBlockProblem(F, dF, (v, p, u, z, b), bc)
F_vec = create_vector_block(F)
dF_mat = create_matrix_block(dF)

In [ ]:
# Solve
vpuzb = create_vector_block(F)
snes = PETSc.SNES().create(mesh.mpi_comm())
snes.setTolerances(max_it=20)
snes.getKSP().setType("preonly")
snes.getKSP().getPC().setType("lu")
snes.getKSP().getPC().setFactorSolverType("mumps")
snes.setObjective(problem.obj)
snes.setFunction(problem.F, F_vec)
snes.setJacobian(problem.dF, J=dF_mat, P=None)
snes.setMonitor(lambda _, it, residual: print(it, residual))
snes.solve(None, vpuzb)
problem.update_solutions(vpuzb)  # TODO can this be safely removed?

In [ ]:
J_controlled = mesh.mpi_comm().allreduce(assemble_scalar(J), op=MPI.SUM)
print("Optimal J =", J_controlled)
assert np.isclose(J_controlled, 9.9127635e-7)

In [ ]:
pyvista_vector_field_plot(mesh, v, "state velocity", factor=1e-1)

In [ ]:
pyvista_scalar_field_plot(mesh, p, "state pressure")

In [ ]:
pyvista_vector_field_plot(mesh, u, "control", factor=1e-1)

In [ ]:
pyvista_vector_field_plot(mesh, z, "adjoint velocity", factor=1e4)

In [ ]:
pyvista_scalar_field_plot(mesh, b, "adjoint pressure")